In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import math
import instance
import Multitours as MT
import IRP_MGSP

# Data

In [3]:
# Read excel file for rows 1 to ncustomers for demanda data
ncustomers = 10
nvehicles = 4
vehicleCapacity = [60000 for i in range(nvehicles)]
Customers = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example3.xlsx', sheet_name='Sheet1', nrows=ncustomers)
Time = pd.read_excel('D:/DIEGO/Gent/2nd Year/Thesis/Example3.xlsx', sheet_name='Sheet1', skiprows=ncustomers+2, usecols="A:C")

Customers["Tmax"] = Customers["Capacity"] / Customers["Demand"]

### Travel Times and other parameters

In [4]:

# Vehicle cost parameter
psi = [1000 for i in range(nvehicles)]

# Transportation cost
delta = 100

# Delivery cost
# column Phi of df Costumers with indeces 1 to ncustomers
phi = Customers["Phi"].copy()
phi.index = phi.index + 1

# Holding Cost
h = Customers["Holding"].copy()
h.index = h.index + 1

# Demand
d = Customers["Demand"].copy()
d.index = d.index + 1

# Capacity
k = Customers["Capacity"].copy()
k.index = k.index + 1

# Time parameter
shape = (ncustomers + 1, ncustomers + 1, nvehicles)
t = np.full(shape, 100)

for v in range(nvehicles):
    for index, row in Time.iterrows():
        i = row['Origin']
        j = row['Destination']
        t[i, j, v] = row['Time']

### Read instance date from files

In [2]:
nvehicles = 5
set_number = 0 
txt_file = "Y15-2A9"
ncustomers, Customers, Time, Vehicles, Network, delta = instance.read_instance(nvehicles, set_number, txt_file)
parameters = instance.instance_parameters(ncustomers, nvehicles, Customers, Time, Vehicles, delta)
ncustomers, nvehicles, psi, delta, phi, h, d, k, t, vehicleCapacity = instance.get_instance_parameters(parameters)

In [3]:
# Time parameter
shape = (ncustomers + 1, ncustomers + 1, nvehicles)
t = np.full(shape, 100.0)

for v in range(nvehicles):
    for index, row in Time.iterrows():
        i = int(row['Origin'])
        j = int(row['Destination'])
        t[i, j, v] = row['Time']

# Model implementation

In [5]:
#model implementation
model = gp.Model('NLIP model')

# Indeces
x_indeces = [(i, j, v) for i in range(ncustomers+1) for j in range(ncustomers+1) for v in range(nvehicles)]
y_indices = [i for i in range(nvehicles)]
Customers_indeces = [i for i in range(1, ncustomers+1)]
Rplus_indices = [i for i in range(ncustomers+1)]
z_indeces = [(i, j, v) for i in Rplus_indices for j in Rplus_indices for v in range(nvehicles)]
vehicle_indeces = [i for i in range(nvehicles)]

#Variables
x = model.addVars(x_indeces, name = 'x', vtype = GRB.BINARY)
y = model.addVars(y_indices, name = 'y', vtype = GRB.BINARY)
z = model.addVars(z_indeces, name = 'z', vtype = GRB.CONTINUOUS, lb=0)
T = model.addVars(vehicle_indeces, name = 'T', vtype = GRB.CONTINUOUS, lb=0, ub=1000)
T_reciprocal = model.addVars(vehicle_indeces, name = 'T_reciprocal', vtype = GRB.CONTINUOUS, lb=1/1000, ub=1/0.0001)
model.setParam("NonConvex", 2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-29
Set parameter NonConvex to value 2


In [6]:
model.setObjective( gp.quicksum(psi[v] * y[v] +
                                  T_reciprocal[v] * (gp.quicksum(gp.quicksum(delta * t[i, j, v] * x[i, j, v] for j in Rplus_indices) for i in Rplus_indices)) +
                                  gp.quicksum((phi[i] * T_reciprocal[v] + 0.5 * h[i] * d[i] * T[v]) * gp.quicksum(x[i, j, v] for j in Rplus_indices) for i in Customers_indeces) for v in vehicle_indeces), GRB.MINIMIZE)

In [7]:
# # Original Objective Function
# model.addConstr(TC == gp.quicksum(psi[v]*y[v] +
#                                   (1/T[v]) * (gp.quicksum(gp.quicksum(delta*t[i,j,v]*x[i,j,v] for j in Rplus_indices) for i in Rplus_indices)) + 
#                                   gp.quicksum((phi[i]*(1/T[v]) + 0.5*h[i]*d[i]*T[v]) * gp.quicksum(x[i,j,v] for j in Rplus_indices) for i in Customers_indeces) for v in vehicle_indeces))

In [8]:
# Constraint 0
for v in vehicle_indeces:
    model.addConstr(T[v] * T_reciprocal[v] == 1, name = 'Reciproco')


# Constraint 1
for j in Customers_indeces:
    model.addConstr(gp.quicksum(x[i,j,v] for i in Rplus_indices for v in vehicle_indeces) == 1, name = '1')

# Constraint 2
for j in Rplus_indices:
    for v in vehicle_indeces:
        model.addConstr(gp.quicksum(x[i,j,v] for i in Rplus_indices if i != j) - gp.quicksum(x[j,k,v] for k in Rplus_indices if j!=k) == 0, name = '2')

# Constraint 3
for v in vehicle_indeces:
    model.addConstr(gp.quicksum(x[i,j,v] * t[i,j,v] for j in Rplus_indices for i in Rplus_indices if i!=j) - T[v] * y[v] <= 0, name = '3')

# Constraint 4
for j in Customers_indeces:
    model.addConstr(gp.quicksum(z[i,j,v] for i in Rplus_indices for v in vehicle_indeces if i!=j) - gp.quicksum(z[j,k,v] for k in Rplus_indices for v in vehicle_indeces if j!=k) == d[j], name = '4')

# Constraint 5
for j in Customers_indeces:
    for v in vehicle_indeces:
        model.addConstr(T[v]*z[0,j,v] <= vehicleCapacity[v], name = '5')

# Constraint 6
for j in Customers_indeces:
    for v in vehicle_indeces:
        model.addConstr(T[v]*d[j] * gp.quicksum( x[i,j,v] for i in Customers_indeces) <= k[j], name = '6')

# Constraint 7
for i in Rplus_indices:
    for j in Rplus_indices:
        if i!=j:
            for v in vehicle_indeces:
                model.addConstr(z[i,j,v] <= x[i,j,v]*gp.quicksum(d[k] for k in Customers_indeces), name = '7')


In [9]:
#Solve
model.update()

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 504 rows, 980 columns and 3000 nonzeros
Model fingerprint: 0xd192cfb5
Model has 924 quadratic objective terms
Model has 88 quadratic constraints
Variable types: 492 continuous, 488 integer (488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [1e+00, 2e+03]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [1e+03, 1e+03]
  QObjective range [2e+02, 2e+04]
  Bounds range     [1e-03, 1e+04]
  RHS range        [1e+00, 2e+03]
  QRHS range       [1e+00, 7e+04]
Presolve removed 0 rows and 88 columns
Presolve time: 0.03s
Presolved: 1932 rows, 2616 columns, 7696 nonzeros
Presolved model has 880 SOS constraint(s)
Presolved model has 44 bilinear constraint(s)

Solving non-convex MIQCP

Variable 

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "d:\AnacondaDiego\envs\new\lib\site-packages\ipykernel\iostream.py", line 526, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 1977017 961640     cutoff  148      127393.326 73307.9960  42.5%  53.2 6818s
 1977892 962181 99537.2421  202   24 127393.326 73310.1859  42.5%  53.2 6825s
 1979086 962434 101613.530  161   22 127393.326 73312.6305  42.5%  53.2 6831s
 1979983 962740 122394.093  192   26 127393.326 73314.0849  42.5%  53.2 6836s
 1980788 963078     cutoff  197      127393.326 73315.0517  42.4%  53.2 6842s
 1981697 963448 78690.5106  119   25 127393.326 73318.1493  42.4%  53.2 6847s


In [ ]:
for v in model.getVars():
    if v.x != 0:
        print(f"{v.varName}: {v.x}")
print(f"Obj: {model.objVal}")

x[0,2,0]: 1.0
x[0,5,0]: 1.0
x[0,7,0]: 1.0
x[0,8,1]: 1.0
x[0,9,1]: 1.0
x[0,10,1]: 1.0
x[0,12,1]: 1.0
x[0,13,0]: 1.0
x[0,15,1]: 1.0
x[1,0,0]: 1.0
x[2,6,0]: 1.0
x[3,0,0]: 1.0
x[4,0,1]: 1.0
x[5,3,0]: 1.0
x[6,0,0]: 1.0
x[7,1,0]: 1.0
x[8,0,1]: 1.0
x[9,0,1]: 1.0
x[10,4,1]: 1.0
x[11,0,1]: 1.0
x[12,11,1]: 1.0
x[13,0,0]: 1.0
x[14,0,1]: 1.0
x[15,14,1]: 1.0
y[0]: 1.0
y[1]: 1.0
z[0,2,0]: 51.4
z[0,5,0]: 43.0
z[0,7,0]: 35.8
z[0,8,1]: 27.1
z[0,9,1]: 19.7
z[0,10,1]: 35.3
z[0,12,1]: 41.4
z[0,13,0]: 29.4
z[0,15,1]: 33.8
z[1,0,0]: -3.552713678800501e-15
z[2,6,0]: 26.0
z[4,0,1]: -3.552713678800501e-15
z[5,3,0]: 29.9
z[7,1,0]: 22.299999999999997
z[10,4,1]: 15.799999999999997
z[12,11,1]: 20.9
z[14,0,1]: -3.552713678800501e-15
z[15,14,1]: 18.799999999999997
T[0]: 1.8834612359550567
T[1]: 2.403
T_reciprocal[0]: 0.5309373938311636
T_reciprocal[1]: 0.4161464835622139
Obj: 2984.774573320237


In [ ]:
solution = {}
for index_tuple, value in x.items():
    if value.x > 1e-5:
        if index_tuple[2] not in solution:
            solution[index_tuple[2]] = []
        solution[index_tuple[2]].append(index_tuple[:2])

for v in solution.keys():
    multitour = IRP_MGSP.edge_to_multitour(solution[v])
    print(multitour)